Student Name: Manoj Chandra Machavolu<br>
Student Number: 1008518506

In [0]:
#all spark imports
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
spark = SparkSession.builder.appName("Integer").getOrCreate()

**Part A Q1)**

In [0]:
integerRDD = spark.sparkContext.textFile("/FileStore/tables/integer_1.txt", 8)

In [0]:
integerRDD.count()

Out[4]: 1010

In [0]:
newRDD = integerRDD.map(lambda x: float(x))

In [0]:
print('The total number of even numbers = ',newRDD.filter(lambda x: x % 2 == 0).count())

The total number of even numbers =  514


In [0]:
print('The total number of odd numbers = ',newRDD.filter(lambda x: x % 2 != 0).count())

The total number of odd numbers =  496


**Part A Q2)**

In [0]:
salaryRDD = spark.sparkContext.textFile("/FileStore/tables/salary_1.txt", 4)

In [0]:
salaryMapRDD = salaryRDD.map(lambda x: x.split(" "))

In [0]:
print("Salary sum per department")
mappedRDD = salaryMapRDD.map(lambda x: (x[0], int(x[1])))
sumRDD = mappedRDD.reduceByKey(lambda x,y: x+y)
sumRDD.collect()

Salary sum per department
Out[10]: [('Developer', 3221394),
 ('Sales', 3488491),
 ('Research', 3328284),
 ('QA', 3360624),
 ('Marketing', 3158450)]

**Part A Q3)**

In [0]:
textFile = spark.read.text("/FileStore/tables/shakespeare_2.txt")

In [0]:
print('As per the question, only exact words were counted')
print('Shakespeare ',textFile.rdd.map(lambda x: x[0].split(' ').count('Shakespeare')).sum())
print('why ',textFile.rdd.map(lambda x: x[0].split(' ').count('why')).sum())
print('Lord ',textFile.rdd.map(lambda x: x[0].split(' ').count('Lord')).sum())
print('Library ',textFile.rdd.map(lambda x: x[0].split(' ').count('Library')).sum())
print('GLUTENBERG ',textFile.rdd.map(lambda x: x[0].split(' ').count('GUTENBERG')).sum())
print('WILLIAM ',textFile.rdd.map(lambda x: x[0].split(' ').count('WILLIAM')).sum())
print('COLLEGE ',textFile.rdd.map(lambda x: x[0].split(' ').count('COLLEGE')).sum())
print('WORLD ',textFile.rdd.map(lambda x: x[0].split(' ').count('WORLD')).sum())

As per the question, only exact words were counted
Shakespeare  22
why  91
Lord  341
Library  2
GLUTENBERG  99
WILLIAM  115
COLLEGE  98
WORLD  98


**Part A Q4)**

In [0]:
shakespeare_2RDD = spark.sparkContext.textFile("/FileStore/tables/shakespeare_2.txt").flatMap(lambda line: line.split())

In [0]:
mapRDD = shakespeare_2RDD.map(lambda word: (word, 1))
resultRDD = mapRDD.reduceByKey(lambda a,b:a+b)

In [0]:
print('The top 10 most occuring words with their counts are ',resultRDD.map(lambda x: (x[1], x[0])).sortByKey(False).take(11))

The top 10 most occuring words with their counts are  [(11397, 'the'), (8777, 'and'), (8556, 'I'), (7873, 'of'), (7421, 'to'), (5672, 'a'), (4913, 'my'), (4600, 'in'), (4060, 'you'), (3547, 'And'), (3522, 'that')]


In [0]:
print('The top 10 least occuring words with their counts are ',resultRDD.map(lambda x: (x[1], x[0])).sortByKey(True).take(10))

The top 10 least occuring words with their counts are  [(1, 'anyone'), (1, 'restrictions'), (1, 'whatsoever.'), (1, 're-use'), (1, 'online'), (1, 'www.gutenberg.org'), (1, 'COPYRIGHTED'), (1, 'eBook,'), (1, 'Details'), (1, 'guidelines')]


**Part B**

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.feature import Imputer, StringIndexer, VectorAssembler
from pyspark.ml.linalg import SparseVector, DenseVector
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [0]:
spark = SparkSession.builder.appName("Recommender systems").getOrCreate()

#set the shuffle partition same as number of cpu cores to improve performance 
spark.conf.set("spark.sql.shuffle.partitions", 4)

In [0]:
path = "/FileStore/tables/movies_1.csv"

df = spark.read \
  .format("csv") \
  .option("inferSchema", True) \
  .option("header", True) \
  .option("sep", ',') \
  .option("path", path) \
  .load()

The csv file consists of 3 columns namely movieId, rating and userId.<br>The ratings are out of 5 (5 means very good movie and 1 means poor).<br>

**Part B Q1)**

In [0]:
print("Top 20 movies with highest ratings")
new_df = df.groupBy("movieId").agg(count("rating")).orderBy(col("count(rating)").desc())
new_df.show(20)

Top 20 movies with highest ratings
+-------+-------------+
|movieId|count(rating)|
+-------+-------------+
|     51|           20|
|     50|           20|
|      6|           20|
|     29|           20|
|     22|           20|
|     15|           19|
|     55|           19|
|     68|           19|
|      2|           19|
|     94|           19|
|     86|           18|
|     45|           18|
|     36|           18|
|     14|           18|
|     88|           18|
|     85|           18|
|     19|           17|
|     12|           17|
|     67|           17|
|     56|           17|
+-------+-------------+
only showing top 20 rows



In [0]:
print("Top 15 users who provided the highest ratings")
new_df_2 = df.groupBy("userId").agg(count("rating")).orderBy(col("count(rating)").desc())
new_df_2.show(15)

Top 15 users who provided the highest ratings
+------+-------------+
|userId|count(rating)|
+------+-------------+
|    14|           57|
|     6|           57|
|    22|           56|
|    11|           56|
|    12|           55|
|     4|           55|
|     7|           54|
|     9|           53|
|    23|           52|
|    24|           52|
|    18|           52|
|    28|           50|
|    19|           49|
|     1|           49|
|     5|           49|
+------+-------------+
only showing top 15 rows



**Part B Q2)**

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator

Combination 1: 60% Train set and 40% Test set

In [0]:
(training_1, test_1) = df.randomSplit([0.6, 0.4], 20)   #### Setting a constant seed value so that the split remais constant throughout

In [0]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als_1 = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model_1 = als_1.fit(training_1)

In [0]:
# Evaluate the model by computing the RMSE on the test data
predictions_1 = model_1.transform(test_1)

Combination 2: 80% Train set and 20% Test set

In [0]:
(training_2, test_2) = df.randomSplit([0.8, 0.2], 20)   #### Setting a constant seed value so that the split remais constant throughout

In [0]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als_2 = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model_2 = als_2.fit(training_2)

In [0]:
predictions_2 = model_2.transform(test_2)

**Part B Q3)**

**MAE - Mean Absolute Error**<br>
MAE evaluates the absolute distance of the observations (the entries of the dataset) to the predictions on a regression, taking the average over all observations. We use the absolute value of the distances so that negative errors are accounted properly.<br>
Formula for MAE - https://miro.medium.com/max/510/1*V5kDRZyG6udD7tKStFotlQ.png<br>
**MSE - Mean Squared Error**<br>
MSE squares the distance between the actual and prediction values, taking the average over all observations. Higher errors (or distances) weigh more in the metric than lower ones, due to the nature of the power function. A backlash in MSE is the fact that the unit of the metric is also squared, so if the model tries to predict price in US$, the MSE will yield a number with unit (US$)² which does not make sense.<br>
Formula for MSE - https://miro.medium.com/max/532/1*VZ3iKUrzH7f71B03Sui6fQ.png<br>
**RMSE - Root Mean Squared Error**<br>
RMSE is used to return the MSE error to the original unit by taking the square root of it, while maintaining the property of penalizing higher errors.<br>
Formula for RMSE -  https://miro.medium.com/max/640/1*YDOgUMYYjETdAVAn7xEOrw.png

**MAE and RMSE — Which Metric is Better?**  Link - https://medium.com/human-in-a-machine-world/mae-and-rmse-which-metric-is-better-e60ac3bde13d<br>RMSE has the benefit of penalizing large errors more so can be more appropriate in some cases.<br>Thus we prefer RMSE over MAE and MSE

In [0]:
# Evaluate the model by computing the RMSE on the test data 1
evaluator_1 = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse_1 = evaluator_1.evaluate(predictions_1)
print("Root-mean-square error = " + str(rmse_1))

Root-mean-square error = 1.1427514549626354


In [0]:
# Evaluate the model by computing the RMSE on the test data 2
evaluator_2 = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse_2 = evaluator_2.evaluate(predictions_2)
print("Root-mean-square error = " + str(rmse_2))

Root-mean-square error = 1.0365173812132722


Thus, the dataset with train test split of 80-20 is effective and has less errors since a larger part of data (80%) is used for training the model compared to only 60%.

**Part B Q4)**

Hyperparameter tuning of model 1

In [0]:
paramGrid = ParamGridBuilder()\
    .addGrid(als_1.regParam, [0.1, 0.01]) \
    .addGrid(als_1.maxIter, [3,4,5])\
    .addGrid(als_1.alpha, [0.4,0.8])\
    .build()

In [0]:
crossval_1 = CrossValidator(estimator=als_1,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator_1,
                           numFolds=3)

In [0]:
model_1 = crossval_1.fit(training_1)

In [0]:
model_1_prediction = model_1.transform(test_1)
rmse_1_tuned = evaluator_1.evaluate(model_1_prediction)
print("Root-mean-square error = " + str(rmse_1_tuned))

Root-mean-square error = 1.2186136086995472


In [0]:
import numpy as np
print(model_1.getEstimatorParamMaps()[np.argmax(model_1.avgMetrics)])

{Param(parent='ALS_692ec9eeafa9', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='ALS_692ec9eeafa9', name='maxIter', doc='max number of iterations (>= 0).'): 3, Param(parent='ALS_692ec9eeafa9', name='alpha', doc='alpha for implicit preference'): 0.4}


Thus the best parameters for model 1 (60/40 split) are : regParam = 0.01, maxIter = 3 and alpha = 0.4<br>The RMSE of model 1 after tuning is 1.2186

Hyperparameter tuning of model 2

In [0]:
crossval_2 = CrossValidator(estimator=als_2,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator_2,
                           numFolds=3)

In [0]:
model_2 = crossval_2.fit(training_2)

In [0]:
model_2_prediction = model_2.transform(test_2)
rmse_2_tuned = evaluator_2.evaluate(model_2_prediction)
print("Root-mean-square error = " + str(rmse_2_tuned))

Root-mean-square error = 1.0365173812132722


In [0]:
print(model_2.getEstimatorParamMaps()[np.argmax(model_2.avgMetrics)])

{Param(parent='ALS_692ec9eeafa9', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent='ALS_692ec9eeafa9', name='maxIter', doc='max number of iterations (>= 0).'): 3, Param(parent='ALS_692ec9eeafa9', name='alpha', doc='alpha for implicit preference'): 0.4}


Thus the best parameters for model 2 (80/20 split) are regParam = 0.1, maxIter = 4 and alpha = 0.8<br>The RMSE of model 2 after tuning is 1.0365 which is same as before tuning.<br> Hence my recommendation is to not tune the hyper parameters as it just leads to increase in computation time.

**Comparing the 2 models after Hyper parameter tuning, the best model is model 2 with 80/20 split ratio of RMSE 1.0365**<br>
More the data used for training, better is the prediction

**Significance of parameters used for Hyper tuning.**<br>
**maxIter** is the maximum number of iterations to run (defaults to 10).<br>
**regParam** specifies the regularization parameter in ALS (defaults to 1.0).<br>
**alpha** is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations (defaults to 1.0).

**Part 2 Q5)**

Now, extracting the best hyper paramters and using model 2 in ALS, we get

In [0]:
als = ALS(maxIter=4, regParam=0.1, alpha=0.8, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training_2)

In [0]:
predictions = model.transform(test_2)

In [0]:
userRecs = model.recommendForAllUsers(15)
userRecs.display()

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


userId,recommendations
20,"List(List(22, 3.5436418), List(77, 3.1543994), List(88, 2.9691494), List(62, 2.691823), List(51, 2.6850338), List(74, 2.6340606), List(7, 2.4530618), List(98, 2.38503), List(54, 2.3844337), List(94, 2.3465044), List(24, 2.293388), List(32, 2.1524203), List(92, 2.1471808), List(87, 2.1471012), List(41, 2.0527668))"
10,"List(List(49, 2.9692636), List(40, 2.7998114), List(92, 2.5178447), List(42, 2.4445102), List(47, 2.202442), List(4, 2.197721), List(25, 2.1546082), List(0, 2.0897472), List(2, 2.0821645), List(81, 2.0391595), List(85, 1.9937572), List(89, 1.960944), List(1, 1.8860781), List(93, 1.7398272), List(12, 1.7333003))"
0,"List(List(92, 2.8872342), List(62, 2.2493389), List(2, 2.1866317), List(49, 1.9753945), List(89, 1.9657198), List(1, 1.9413546), List(12, 1.9109589), List(4, 1.826584), List(22, 1.8096465), List(77, 1.7510881), List(95, 1.7288542), List(91, 1.7239553), List(41, 1.7123939), List(57, 1.6924877), List(81, 1.6646932))"
1,"List(List(62, 2.7366626), List(77, 2.6835294), List(98, 2.559925), List(29, 2.497772), List(90, 2.429508), List(28, 2.414606), List(85, 2.4139247), List(94, 2.3954282), List(22, 2.3829741), List(52, 2.3825567), List(74, 2.297678), List(51, 2.2233999), List(92, 2.2078123), List(41, 2.1721256), List(88, 2.0996337))"
11,"List(List(27, 4.6242313), List(23, 4.527977), List(32, 4.514224), List(69, 4.504795), List(48, 4.256778), List(18, 4.1610374), List(79, 4.1162205), List(90, 4.075428), List(34, 3.770828), List(66, 3.6660514), List(19, 3.6472616), List(13, 3.576232), List(81, 3.5759928), List(38, 3.4879177), List(30, 3.3979263))"
21,"List(List(29, 4.41662), List(52, 4.207973), List(53, 3.8503525), List(62, 3.808525), List(74, 3.487352), List(76, 3.437942), List(41, 3.073553), List(70, 2.8862903), List(63, 2.8537436), List(85, 2.8116713), List(58, 2.7631187), List(72, 2.720936), List(93, 2.7039273), List(96, 2.5343096), List(67, 2.5243573))"
12,"List(List(27, 4.7341595), List(32, 4.4351935), List(35, 4.3454323), List(17, 4.1923122), List(55, 4.000731), List(48, 3.9757478), List(64, 3.9035752), List(30, 3.903024), List(49, 3.8981318), List(90, 3.8806548), List(46, 3.8727002), List(50, 3.5768325), List(16, 3.539682), List(94, 3.4145155), List(31, 3.0832968))"
2,"List(List(83, 4.499529), List(93, 4.4600716), List(39, 4.053454), List(37, 3.9827049), List(92, 3.8548613), List(40, 3.5830336), List(89, 3.5568874), List(34, 3.492097), List(19, 3.4342837), List(81, 3.2545238), List(4, 3.0386217), List(49, 3.0089474), List(28, 3.001029), List(23, 2.9694078), List(43, 2.9105105))"
22,"List(List(22, 4.464737), List(74, 4.428699), List(88, 4.31415), List(77, 3.6875594), List(32, 3.5696619), List(62, 3.5036075), List(7, 3.460813), List(51, 3.3258634), List(98, 3.2181628), List(54, 3.213729), List(24, 3.1487114), List(41, 3.025909), List(53, 2.9823115), List(29, 2.977372), List(94, 2.963779))"
13,"List(List(74, 2.774088), List(29, 2.5024345), List(41, 2.463862), List(53, 2.4456398), List(52, 2.416337), List(62, 2.4118762), List(93, 2.3580184), List(92, 2.2555149), List(70, 2.1916344), List(28, 2.0996432), List(7, 2.0223842), List(39, 2.006426), List(87, 1.9602845), List(37, 1.9282062), List(83, 1.9176053))"


In [0]:
import pyspark.sql.functions as f
userRecs.filter(f.col('userId')==10).display()

userId,recommendations
10,"List(List(49, 2.9692636), List(40, 2.7998114), List(92, 2.5178447), List(42, 2.4445102), List(47, 2.202442), List(4, 2.197721), List(25, 2.1546082), List(0, 2.0897472), List(2, 2.0821645), List(81, 2.0391595), List(85, 1.9937572), List(89, 1.960944), List(1, 1.8860781), List(93, 1.7398272), List(12, 1.7333003))"


In [0]:
userRecs.filter(f.col('userId')==14).display()

userId,recommendations
14,"List(List(29, 4.7187376), List(52, 4.655295), List(76, 4.1710854), List(85, 4.0850472), List(62, 3.7639232), List(96, 3.279928), List(63, 3.1719012), List(72, 3.1679373), List(69, 2.9123392), List(53, 2.908828), List(90, 2.886309), List(47, 2.8856504), List(30, 2.8075178), List(58, 2.7764516), List(67, 2.7576265))"


There is a chance that the movie recommended to a user is already watched and rated by that user.<br> Hence we need to exclude the movieId already rated by the user in order to get new recommendations.

In [0]:
userData = df.filter(df.userId.isin([10,14]))
userSubsetRecs = model.recommendForUserSubset(userData, 100)

In [0]:
newRecommendationsDF = (userSubsetRecs.select('userId',explode('recommendations').alias('recommendation')).select('userId','recommendation.*')).sort('rating',ascending=False)
newRecommendationsDF.select('movieId','rating','userId')

Out[46]: DataFrame[movieId: int, rating: float, userId: int]

In [0]:
print("Top 15 movies recommended to user 10 are ")
result1 = newRecommendationsDF.join(userData, on =['movieId'],how ='left_anti')
result1.filter(result1.userId.isin([10])).show(15)

Top 15 movies recommended to user 10 are 
+-------+------+----------+
|movieId|userId|    rating|
+-------+------+----------+
|     92|    10| 2.5178447|
|     81|    10| 2.0391595|
|     85|    10| 1.9937572|
|     91|    10| 1.6262096|
|     87|    10| 1.5756335|
|     57|    10| 1.5597557|
|     19|    10| 1.3917103|
|     33|    10|  1.305766|
|     61|    10| 1.2916336|
|     65|    10| 1.1958983|
|     70|    10| 1.1757578|
|     23|    10| 0.9971097|
|     30|    10|0.98390806|
|     99|    10| 0.9517034|
|     64|    10| 0.9479872|
+-------+------+----------+
only showing top 15 rows



In [0]:
print("Top 15 movies recommended to user 14 are ")
result2 = newRecommendationsDF.join(userData, on =['movieId'],how ='left_anti')
result2.filter(result2.userId.isin([14])).show(15)

Top 15 movies recommended to user 14 are 
+-------+------+---------+
|movieId|userId|   rating|
+-------+------+---------+
|     85|    14|4.0850472|
|     30|    14|2.8075178|
|     59|    14|2.6735032|
|     70|    14|2.4229238|
|     74|    14|2.3021817|
|     87|    14|2.1188734|
|     61|    14|1.9486207|
|     60|    14|1.7559137|
|     92|    14|1.6789422|
|     91|    14| 1.457192|
|     32|    14|1.4171797|
|     81|    14|1.2478681|
|     26|    14|1.1856362|
|     46|    14|1.1693587|
|     33|    14|1.1266334|
+-------+------+---------+
only showing top 15 rows

